# OpenDota → Supabase → Streamlit (Colab/Juniper)
Pipeline básico para baixar partidas de um campeonato, gerar XML e popular o Supabase.

In [ ]:
# 1) Instalar dependências mínimas
%pip install --quiet requests pandas supabase psycopg2-binary streamlit

In [ ]:
# 2) Configurar variáveis sensíveis
import os
os.environ['OPENDOTA_API_KEY'] = '00495232-b2b4-4d0b-87e3-c01de846c4b4'  # substitua se necessário
os.environ['SUPABASE_URL'] = 'https://<sua-instancia>.supabase.co'
os.environ['SUPABASE_SERVICE_KEY'] = '<chave-service-role>'
os.environ['SUPABASE_DB_URL'] = 'postgresql://.../postgres'  # conexão direta para DDL
LEAGUE_ID = 0  # troque para o ID do campeonato em uso

In [ ]:
# 3) Ingestão OpenDota (baixa XML + JSON normalizado)
from pathlib import Path
import sys

repo_dir = Path('..').resolve()
sys.path.append(str(repo_dir / '03_INFRAESTRUTURA'))

from app.data.opendota.ingest import run_ingestion

output = run_ingestion(league_id=LEAGUE_ID, output_dir='data/opendota_colab')
output

In [ ]:
# 4) Aplicar schema no Supabase e carregar dados
from app.data.opendota import SupabaseLoader, SupabaseSchemaManager

schema = SupabaseSchemaManager()
schema.apply()

loader = SupabaseLoader()
loader.load_normalized(output['normalized_file'])
